<a href="https://colab.research.google.com/github/yoerisamwel/code_notes/blob/main/practicing_parquet_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [665 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,685 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,060 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,398 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 https://p

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [3]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/Austin_311_Public_Data.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Austin_311_Public_Data.csv"), sep=",", header=True)
df.show()


+----------------------+--------------------+------------------+------+--------------+--------------+--------------------+-------------+--------------------+------+--------+----------+----------------+
|service_request_number|         description|   method_received|status|  created_date|   closed_date|            location|street_number|         street_name|  city|zip_code|    county|council_dsitrict|
+----------------------+--------------------+------------------+------+--------------+--------------+--------------------+-------------+--------------------+------+--------+----------+----------------+
|           19-00090956|Animal - Proper Care|             Phone|Closed| 3/14/19 15:02|11/23/20 13:41|4609 RIBBECKE AVE...|         4609|            RIBBECKE|AUSTIN|   78721|    TRAVIS|               1|
|           20-00135805|      Tree Issue ROW|     Mobile Device|Closed|  4/7/20 19:06| 11/23/20 0:02|3521 WEST AVE, AU...|         3521|                WEST|AUSTIN|   78705|    TRAVIS|        

In [4]:
# Lets create a view with our DataFrame that counts the occurances of the service request description of the 311 issues.
# Record the runtime using time.time() method.
df.createOrReplaceTempView('description_view')
start_time = time.time()

spark.sql("""select description, count(description) from description_view group by description""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+------------------+
|         description|count(description)|
+--------------------+------------------+
|Tree Issue ROW/Ma...|              4688|
|Austin Code - Sho...|               848|
|Park Maintenance ...|              2690|
|Austin Code - Sho...|              9865|
|Lane/Road Closure...|              9836|
|ZZZ ARR Missed Re...|                20|
|ATD - Booting Com...|               245|
|  Obstruction in ROW|              9453|
|Park Maintenance ...|              1646|
|Parking Ticket Co...|               407|
|Parking Machine I...|             10756|
|School Zone Flash...|              2956|
|ATD - Shared Micr...|               717|
|Residential Parki...|               153|
|       Bat Complaint|              1074|
|AW - Water Conser...|                58|
|Street Light Issu...|              2052|
|Austin Code - Con...|               125|
|Channels/Creeks/D...|              5410|
|   Barricade Request|               228|
+--------------------+------------

In [5]:
# Write out the data in parquet format
df.write.parquet('description_parket')



*   click the folder icon on the left and notice that a new folder is present with the same name as your parquet file
*   inside of it you will find 'part_x.parquet' files
*  these are binary files that store your compressed data in columnar format





In [6]:
# Read in our new parquet formatted data
p_df=spark.read.parquet('description_parket')

In [7]:
# A parquet formatted DataFrame has all the same methods as a row-based DataFrame
# we can convert the DataFrame to a view.
p_df.createOrReplaceTempView('description_view')

In [8]:
# Run the same sql as above.  (Note: Due to small datasets it IS possible that times may be very close, however this is not the case in larger sets.)
start_time = time.time()

spark.sql("""select description, count(description) from description_view group by description""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+------------------+
|         description|count(description)|
+--------------------+------------------+
|Tree Issue ROW/Ma...|              4688|
|Austin Code - Sho...|               848|
|Park Maintenance ...|              2690|
|Austin Code - Sho...|              9865|
|Lane/Road Closure...|              9836|
|ZZZ ARR Missed Re...|                20|
|ATD - Booting Com...|               245|
|  Obstruction in ROW|              9453|
|Park Maintenance ...|              1646|
|Parking Ticket Co...|               407|
|Parking Machine I...|             10756|
|School Zone Flash...|              2956|
|ATD - Shared Micr...|               717|
|Residential Parki...|               153|
|       Bat Complaint|              1074|
|AW - Water Conser...|                58|
|Street Light Issu...|              2052|
|Austin Code - Con...|               125|
|Channels/Creeks/D...|              5410|
|   Barricade Request|               228|
+--------------------+------------